In [1]:
import aisuite as ai
import click
import os
from dotenv import load_dotenv
load_dotenv()
from vllm import LLM, SamplingParams
import json

# Huggingface

In [2]:
llm = LLM(model="ToolBench/ToolLLaMA-2-7b-v2", max_model_len=4096)
tokenizer = llm.get_tokenizer()

INFO 01-06 22:13:15 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 01-06 22:13:19 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 01-06 22:13:19 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='ToolBench/ToolLLaMA-2-7b-v2', speculative_config=None, tokenizer='ToolBench/ToolLLaMA-2-7b-v2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collec

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

INFO 01-06 22:13:20 selector.py:135] Using Flash Attention backend.
INFO 01-06 22:13:21 model_runner.py:1072] Starting to load model ToolBench/ToolLLaMA-2-7b-v2...
INFO 01-06 22:13:21 weight_utils.py:243] Using model weights format ['*.bin']


Loading pt checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


/home/kyochul/anaconda3/envs/hcclab/lib/python3.11/site-packages/vllm/model_executor/model_loader/weight_utils.py:425: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = 

INFO 01-06 22:13:36 model_runner.py:1077] Loading model weights took 12.5523 GB
INFO 01-06 22:13:37 worker.py:232] Memory profiling results: total_gpu_memory=23.55GiB initial_memory_usage=13.02GiB peak_torch_memory=12.91GiB memory_usage_post_profile=13.06GiB non_torch_memory=0.50GiB kv_cache_size=7.79GiB gpu_memory_utilization=0.90
INFO 01-06 22:13:37 gpu_executor.py:113] # GPU blocks: 997, # CPU blocks: 512
INFO 01-06 22:13:37 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 3.89x
INFO 01-06 22:13:39 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-06 22:13:39 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 0

In [3]:
import json

tool_list = json.load(open("../temp_tool_desc.json", "r"))["nodes"]

for tool in tool_list:
    parameter_list = []
    for parameter in tool["parameters"]:
        parameter_list.append({
            "name": parameter["name"],
            "description": parameter["desc"]
        })
    tool["parameters"] = parameter_list

In [4]:
# Tool documentation 생성 함수
def generate_tool_document(tool_list):
    document = "TOOL DOCUMENTATION\n"
    document += "Below is a list of tools with their descriptions and required parameters.\n"

    for tool in tool_list:
        document += f"TOOL ID: {tool['id']}\n"
        document += f"DESCRIPTION: {tool['desc']}\n"
        document += "PARAMETERS:\n"
        for param in tool['parameters']:
            document += f"  - {param['name']} ({param.get('type', 'string')}): {param['description']}\n"
        document += "\n"  # 각 도구를 구분하기 위한 빈 줄 추가

    return document

# Tool documentation 생성
tool_document = generate_tool_document(tool_list)

# 결과 출력
print(tool_document)

TOOL DOCUMENTATION
Below is a list of tools with their descriptions and required parameters.
TOOL ID: get_weather
DESCRIPTION: Get the weather for a specific city and a specific day
PARAMETERS:
  - location (string): The location to get the weather for
  - date (string): The date to get the weather for. The date format should follow the followng: MM-DD.

TOOL ID: get_news_for_topic
DESCRIPTION: Get the news for a specific topic
PARAMETERS:
  - topic (string): The topic to get the news for

TOOL ID: stock_operation
DESCRIPTION: Do a specific operation on a specific stock
PARAMETERS:
  - stock (string): The stock to do the operation on
  - operation (string): The operation to do, eg. buy, sell, short, cover etc.

TOOL ID: book_flight
DESCRIPTION: Book a flight for a specific date, from a specific location to a specific destination
PARAMETERS:
  - date (string): The date to book the flight for. The date format should follow the followng: MM-DD.
  - from (string): The location to book the 

## With Reasoning

In [5]:
prompt = """# TOOL DOCUMENT #:
{{tool_document}}

# GOAL #: 
Using the tools provided in # TOOL DOCUMENT #, generate the required functions and their parameter values to fully address the users' request in # DIALOGUE #, especially the last turn of what the user said. 
Only produce output in the JSON format specified below. **Do not include any additional text, explanations, or content outside the JSON format.**

# OUTPUT FORMAT #: 
```
{
  "functions": [
    {
      "function_name": "name_of_the_function",
      "function_reasoning": "reason_for_selecting_the_function",
      "parameters": [
        {
          "parameter_name": "name_of_the_parameter",
          "parameter_value": "value_of_the_parameter",
          "parameter_type": "type_of_the_parameter (e.g., string, integer, boolean, etc.)",
          "parameter_reasoning": "reason_for_selecting_the_parameter"
        }
      ]
    }
  ]
}
```

# REQUIREMENTS #: 
1. Each function must directly address the users' request in # DIALOGUE # and align with tools in the # TOOL DOCUMENT #.
2. Provide the actual values for the parameters, and functions based on the context of the dialogue whenever possible.
3. Define each parameter with its name, value, type, and a brief explanation of its role directly under "parameter_reasoning".
4. **Only generate output in the JSON format specified in # OUTPUT FORMAT #. Do not include any explanations, additional comments, or additional text outside the JSON.**
5. If there are no functions or parameters required based on the dialogue, return an empty JSON object: `{ "functions": [] }`.

# DIALOGUE #: 
{{dialogue}}

Now generate your result using the specified format excluding any additional text, explanations, or content outside the JSON format:
# RESULT #:
"""

In [6]:
dialogues = []
with open('./examples.jsonl', 'r') as file:
    for line in file:
        line = json.loads(line)
        dialogues.append(line['dialogue'])

In [7]:
# sampling_params = SamplingParams(temperature=0.85, top_p=0.95, max_tokens=1024, repetition_penalty=1.1, stop_token_ids=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")])
sampling_params = SamplingParams(temperature=0.85, top_p=0.95, max_tokens=1024, repetition_penalty=1.1)

outputs = []
for dialogue in dialogues:
    input_prompt = prompt.replace('{{tool_document}}', tool_document).replace('{{dialogue}}', dialogue)

    output = llm.generate(input_prompt, sampling_params)
    outputs.append(output)

Processed prompts: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.55s/it, est. speed input: 400.78 toks/s, output: 56.01 toks/s]


In [8]:
print(outputs[2][0].prompt)

# TOOL DOCUMENT #:
TOOL DOCUMENTATION
Below is a list of tools with their descriptions and required parameters.
TOOL ID: get_weather
DESCRIPTION: Get the weather for a specific city and a specific day
PARAMETERS:
  - location (string): The location to get the weather for
  - date (string): The date to get the weather for. The date format should follow the followng: MM-DD.

TOOL ID: get_news_for_topic
DESCRIPTION: Get the news for a specific topic
PARAMETERS:
  - topic (string): The topic to get the news for

TOOL ID: stock_operation
DESCRIPTION: Do a specific operation on a specific stock
PARAMETERS:
  - stock (string): The stock to do the operation on
  - operation (string): The operation to do, eg. buy, sell, short, cover etc.

TOOL ID: book_flight
DESCRIPTION: Book a flight for a specific date, from a specific location to a specific destination
PARAMETERS:
  - date (string): The date to book the flight for. The date format should follow the followng: MM-DD.
  - from (string): The lo

In [9]:
print(outputs[1][0].outputs[0].text)

{
  "functions": [
    {
      "function_name": "name_of_the_function",
      "function_reasoning": "reason_for_selecting_the_function",
      "parameters": [
        {
          "parameter_name": "name_of_the_parameter",
          "parameter_value": "value_of_the_parameter",
          "parameter_type": "type_of_the_parameter (e.g., string, integer, boolean, etc.)",
          "parameter_reasoning": "reason_for_selecting_the_parameter"
        }
      ]
    }
  ]
}
```

# OUTPUT FORMAT #: 
```
{
  "functions": [
    {
      "function_name": "name_of_the_function",
      "function_reasoning": "reason_for_selecting_the_function",
      "parameters": [
        {
          "parameter_name": "name_of_the_parameter",
          "parameter_value": "value_of_the_parameter",
          "parameter_type": "type_of_the_parameter (e.g., string, integer, boolean, etc.)",
          "parameter_reasoning": "reason_for_selecting_the_parameter"
        }
      ]
    }
  ]
}
```

# DIALOGUE #: 
User A: I’m l

In [10]:
with open('graph_desc.json', 'r') as file:
    data = json.load(file)
print(len(data))

2


In [11]:
len(data['nodes'])

40

In [12]:
len(data['links'])

1560